In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setMaster("local[1]").setAppName("ex35")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("ex35").getOrCreate()

25/01/09 16:59:23 WARN Utils: Your hostname, paloma-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/09 16:59:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/09 16:59:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/09 16:59:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/09 16:59:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
schema = "sensorId STRING, date STRING, PM10 FLOAT"
data = spark.read.csv("/home/paloma/Downloads/jupyter_spark/ex35/input.csv", header=False, schema=schema)
data.show()

+--------+----------+----+
|sensorId|      date|PM10|
+--------+----------+----+
|      s1|2016-01-01|20.5|
|      s2|2016-01-01|30.1|
|      s1|2016-01-02|60.2|
|      s2|2016-01-02|20.4|
|      s1|2016-01-03|60.2|
|      s2|2016-01-03|52.5|
+--------+----------+----+



In [4]:
data = data.rdd # transformando em rdd
# deixando a coluna de float com 1 casa decimal
pm_rounded = data.map(lambda line: (line.sensorId, line.date, round(line.PM10, 1)))
pm_rounded.collect()

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 20.4),
 ('s1', '2016-01-03', 60.2),
 ('s2', '2016-01-03', 52.5)]

In [5]:
# obtendo valores da terceira coluna (posicao 2 de cada tupla) e ao final o valor maximo
# o map vai retornar uma lista com os valores da terceira coluna e o max sera aplicado a ela
max_pm = pm_rounded.map(lambda line: line[2]).max()
print(max_pm)

60.2


In [10]:
# obtendo linhas que possuem o valor max de pm10
lines = pm_rounded.filter(lambda line: line[2] == max_pm)
lines.collect()

[('s1', '2016-01-02', 60.2), ('s1', '2016-01-03', 60.2)]

In [11]:
# obtendo apenas as datas
dates = lines.map(lambda line: line[1])
dates.collect()

['2016-01-02', '2016-01-03']